# Get annual mean metrics from 4xCO2 and piControl and save as text files

Things to be aware of

- EC-Earth3 is the diff of r8i1p1f1 and r1i1p1f1
- MCM-UA is rtmt (not rsdt etc)
- I would really like to avoid iris

**TODO** nice script for downloading CMIP6 data et al

In [1]:
import numpy as np
import iris
import iris.analysis.cartography
import iris.coord_categorisation
from iris.experimental.equalise_cubes import equalise_attributes
import glob
import warnings
from netCDF4 import Dataset
warnings.simplefilter('ignore')

In [2]:
from ar6.utils import mkdir_p

vars = {
    'tas'  : 'air_temperature',
    'rsdt' : 'toa_incoming_shortwave_flux',
    'rsut' : 'toa_outgoing_shortwave_flux',
    'rlut' : 'toa_outgoing_longwave_flux'
}

In [3]:
models = glob.glob('/nfs/b0110/Data/cmip6/*/')
models

models = [
     '/nfs/b0110/Data/cmip6/EC-Earth3/',
     '/nfs/b0110/Data/cmip6/EC-Earth3-Veg/',
     '/nfs/b0110/Data/cmip6/MPI-ESM1-2-LR/',
     '/nfs/b0110/Data/cmip6/NESM3/',
     '/nfs/b0110/Data/cmip6/CIESM/',
     '/nfs/b0110/Data/cmip6/TaiESM/',
     '/nfs/b0110/Data/cmip6/MCM-UA-1-0/',
    '/nfs/b0110/Data/cmip6/KIOST-ESM/'
]

['/nfs/b0110/Data/cmip6/UKESM1-0-LL/',
 '/nfs/b0110/Data/cmip6/CESM2/',
 '/nfs/b0110/Data/cmip6/CanESM5/',
 '/nfs/b0110/Data/cmip6/AWI-ESM-1-1-LR/',
 '/nfs/b0110/Data/cmip6/CESM2-FV2/',
 '/nfs/b0110/Data/cmip6/CESM2-WACCM/',
 '/nfs/b0110/Data/cmip6/CAS-ESM2-0/',
 '/nfs/b0110/Data/cmip6/CESM2-WACCM-FV2/',
 '/nfs/b0110/Data/cmip6/CIESM/',
 '/nfs/b0110/Data/cmip6/CMCC-CM2-SR5/',
 '/nfs/b0110/Data/cmip6/CNRM-CM6-1/',
 '/nfs/b0110/Data/cmip6/EC-Earth3-LR/',
 '/nfs/b0110/Data/cmip6/CNRM-CM6-1-HR/',
 '/nfs/b0110/Data/cmip6/EC-Earth3-Veg-LR/',
 '/nfs/b0110/Data/cmip6/CNRM-ESM2-1/',
 '/nfs/b0110/Data/cmip6/TaiESM1/',
 '/nfs/b0110/Data/cmip6/E3SM-1-0/',
 '/nfs/b0110/Data/cmip6/FGOALS-g3/',
 '/nfs/b0110/Data/cmip6/MCM-UA-1-0/',
 '/nfs/b0110/Data/cmip6/GISS-E2-1-G/',
 '/nfs/b0110/Data/cmip6/GISS-E2-1-G-CC/',
 '/nfs/b0110/Data/cmip6/IPSL-CM6A-LR/',
 '/nfs/b0110/Data/cmip6/CAMS-CSM1-0/',
 '/nfs/b0110/Data/cmip6/MIROC6/',
 '/nfs/b0110/Data/cmip6/MIROC-ES2L/',
 '/nfs/b0110/Data/cmip6/MRI-ESM2-0/',
 '/

In [ ]:
for model in models:
    model_name = model.split('/')[-2]
    print(model_name)
    for variant in glob.glob('/nfs/b0110/Data/cmip6/%s/abrupt-4xCO2/r*i1p*f*' % model_name):
        print(variant)
        nvar_exist = 0
        if model=='MCM-UA-1-0':
            vars = {
                'tas'  : 'air_temperature',
                'rtmt' : 'net_downward_radiative_flux_at_top_of_atmosphere_model',
            }
        else:
            vars = {
                'tas'  : 'air_temperature',
                'rsdt' : 'toa_incoming_shortwave_flux',
                'rsut' : 'toa_outgoing_shortwave_flux',
                'rlut' : 'toa_outgoing_longwave_flux'
            }
        for var_name, standard_name in vars.items():
            test = glob.glob(variant + '/Amon/%s/g*/' % var_name)
            nvar_exist = nvar_exist + len(test) # should be either 0 or 1 depending on whether variable exists
        if nvar_exist==len(vars):
            outdir = '../data_input/cmip6/%s/%s/%s' % tuple([v for v in variant.split('/')[-3:]])
            mkdir_p(outdir) # only mkdir if we have data
            for var_name, standard_name in vars.items():
                cube = iris.load(variant + '/Amon/%s/g*/%s_*.nc' % (var_name, var_name), standard_name)
                equalise_attributes(cube)
                cube = cube.concatenate()[0]
                if not cube.coord('longitude').has_bounds():
                    cube.coord('longitude').guess_bounds()
                if not cube.coord('latitude').has_bounds():
                    cube.coord('latitude').guess_bounds()
                iris.coord_categorisation.add_year(cube, 'time', name='year')
                grid_areas = iris.analysis.cartography.area_weights(cube)
                cube_coll = cube.collapsed(['longitude', 'latitude'], iris.analysis.MEAN, weights=grid_areas).aggregated_by('year', iris.analysis.MEAN).data
                bt = cube.attributes['branch_time_in_parent']
                btu= cube.attributes['parent_time_units']
                cal= cube.coord('time').units.calendar
                fo = open('%s/meta.txt' % outdir, 'w')
                fo.write('branch_time_in_parent: ' + str(cube.attributes['branch_time_in_parent']) + '\n')
                fo.write('parent_time_units:     ' + str(cube.attributes['parent_time_units']) + '\n')
                fo.write('calendar:              ' + str(cube.coord('time').units.calendar) + '\n')
                fo.close() #/Amon/%s/g*/latest/%s_*.nc' % (model, var_name, var_name), standard_name)
                np.savetxt('%s/%s.txt' % (outdir, var_name), cube_coll)

In [ ]:
for model in models:
    model_name = model.split('/')[-2]
    print(model_name)
    for variant in glob.glob('/nfs/b0110/Data/cmip6/%s/piControl/r*i1p*f*' % model_name):
        print(variant)
        nvar_exist = 0
        if model=='MCM-UA-1-0':
            vars = {
                'tas'  : 'air_temperature',
                'rtmt' : 'net_downward_radiative_flux_at_top_of_atmosphere_model',
            }
        else:
            vars = {
                'tas'  : 'air_temperature',
                'rsdt' : 'toa_incoming_shortwave_flux',
                'rsut' : 'toa_outgoing_shortwave_flux',
                'rlut' : 'toa_outgoing_longwave_flux'
            }
        for var_name, standard_name in vars.items():
            test = glob.glob(variant + '/Amon/%s/g*/' % var_name)
            nvar_exist = nvar_exist + len(test) # should be either 0 or 1 depending on whether variable exists
        if nvar_exist==len(vars):
            outdir = '../data_input/cmip6/%s/%s/%s' % tuple([v for v in variant.split('/')[-3:]])
            mkdir_p(outdir) # only mkdir if we have data
            for var_name, standard_name in vars.items():
                cube = iris.load(variant + '/Amon/%s/g*/%s_*.nc' % (var_name, var_name), standard_name)
                equalise_attributes(cube)
                cube = cube.concatenate()[0]
                if not cube.coord('longitude').has_bounds():
                    cube.coord('longitude').guess_bounds()
                if not cube.coord('latitude').has_bounds():
                    cube.coord('latitude').guess_bounds()
                iris.coord_categorisation.add_year(cube, 'time', name='year')
                grid_areas = iris.analysis.cartography.area_weights(cube)
                cube_coll = cube.collapsed(['longitude', 'latitude'], iris.analysis.MEAN, weights=grid_areas).aggregated_by('year', iris.analysis.MEAN).data
                bt = cube.attributes['branch_time_in_parent']
                btu= cube.attributes['parent_time_units']
                cal= cube.coord('time').units.calendar
                fo = open('%s/meta.txt' % outdir, 'w')
                fo.write('branch_time_in_parent: ' + str(cube.attributes['branch_time_in_parent']) + '\n')
                fo.write('parent_time_units:     ' + str(cube.attributes['parent_time_units']) + '\n')
                fo.write('calendar:              ' + str(cube.coord('time').units.calendar) + '\n')
                fo.close() #/Amon/%s/g*/latest/%s_*.nc' % (model, var_name, var_name), standard_name)
                np.savetxt('%s/%s.txt' % (outdir, var_name), cube_coll)